In [2]:
import numpy as np
import open3d as o3d
def visualize_point_cloud_hr(points, colors, save_path=None, width=1920, height=1080):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])  # x,y,z
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Create offscreen window for high-res capture
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=width, height=height, visible=save_path is not None)
    vis.add_geometry(pcd)

    # Set background to black for better contrast
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    opt.point_size = 1.0  # Try increasing for better visibility

    vis.poll_events()
    vis.update_renderer()

    if save_path:
        vis.capture_screen_image(save_path, do_render=True)
        print(f"Saved image to {save_path}")
    else:
        vis.run()

    vis.destroy_window()

In [ ]:
##################
# Visualization Utilities for kitti2voxel - ours
##################
# Define the label to color mapping (BGR format)
label_color_map = {
    -1: [255, 255, 0],
    0: [255, 255, 0],          # road (yellow)
    1: [0, 255, 0],        #  sidewalk (green)
    2: [0, 0, 255],    #  parking(blue)
    3: [0, 255, 255],    #  building(sky blue)
    4: [255, 0, 255],     #  vehicles (pink)
    5: [0, 0, 0],          #  trunk(black)
    6: [255, 0, 0],        #  humans (red)
    7: [100, 0 ,100],      #  poles(purple)
    8: [0, 0, 100],        # outlier(dark blue)
    9: [100, 100, 100],    # terrain(grey) 
}

# Read the binary file
def read_bin_file(file_path):
    # Assuming each point has 11 values: x,y,z,intensity,label,e1,e2,e3,nx,ny,nz
    dtype = np.float32  # adjust if your data uses different precision
    data = np.fromfile(file_path, dtype=dtype)
    # Reshape to N x 11 array CAREFUL: has to be 11xN
    print(data.shape)
    points = data.reshape((11,-1))
    return points.T #Nx11

# Add color based on label
def add_color_to_points(points):
    colors = []
    #print(points[:,4])
    for point in points:
        if np.isnan(point[4]):
            label = -1  # or some fallback
        else:
            label = int(point[4])

        #label = int(point[4])  # label is at index 4
        # Get color from map, default to black if label not found
        bgr_color = label_color_map.get(label, [255, 0, 0])
        # Convert BGR to RGB
        rgb_color = [bgr_color[2]/255.0, bgr_color[1]/255.0, bgr_color[0]/255.0]
        colors.append(rgb_color)
    return np.array(colors)

# Main processing
def process_and_visualize(file_path):
    # Read points
    points = read_bin_file(file_path)
    
    # Add colors
    colors = add_color_to_points(points)
    
    # Visualize
    visualize_point_cloud_hr(points, colors, save_path="point_cloud_ours_hr.png", width=1024, height=768)

# Example usage
if __name__ == "__main__":
    file_path = "01/000000.bin"  # Replace with your file path
    process_and_visualize(file_path)


(1882463,)
Saved image to point_cloud_ours_hr.png


In [ ]:
######
# submaps Vislualization Utilities for kitti2voxel - theirs
######

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def create_legend_image(label_color_map, filename="legend.png"):
    # Convert BGR to RGB for plotting
    legend_patches = []
    for label, bgr in label_color_map.items():
        rgb = [bgr[2]/255.0, bgr[1]/255.0, bgr[0]/255.0]
        patch = mpatches.Patch(color=rgb, label=str(label))
        legend_patches.append(patch)

    # Create figure
    fig = plt.figure(figsize=(10, 8))
    plt.legend(handles=legend_patches, loc='center', fontsize='small', title="Labels", ncol=4)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Legend saved to {filename}")

# Define the label to color mapping (BGR format)
label_color_map = {
    0: [255, 255, 255],  # unlabeled / ignore
    1: [0, 0, 255],
    10: [255, 0, 255],
    11: [255, 0, 255],
    13: [250, 80, 100],
    15: [255, 0, 255],
    16: [255, 0, 0],
    18: [255, 0, 255],
    20: [255, 0, 255],
    30: [255, 0, 0],
    31: [255, 0, 0],
    32: [255, 0, 0],
    40: [255, 255, 0],
    44: [0, 0, 255],
    48: [0, 255, 0],
    49: [255, 255, 0],
    50: [0, 255, 255],
    51: [100, 0, 100],
    52: [0, 255, 255],
    60: [100, 0, 100],
    70: [0, 175, 0],
    71: [0, 0, 0],
    72: [100, 100, 100],
    80: [100, 0, 100],
    81: [100, 0, 100],
    99: [0, 0, 100],
    252: [245, 150, 100],
    253: [200, 40, 255],
    254: [30, 30, 255],
    255: [90, 30, 150],
    256: [255, 0, 0],
    257: [250, 80, 100],
    258: [180, 30, 80],
    259: [255, 0, 0]
}

# Read the binary file
def read_bin_file(file_path):
    # Assuming each point has 11 values: x,y,z,intensity,label,e1,e2,e3,nx,ny,nz
    dtype = np.float32  # adjust if your data uses different precision
    data = np.fromfile(file_path, dtype=dtype)
    # Reshape to N x 11 array CAREFUL: has to be 11xN
    print(data.shape)
    points = data.reshape((11,-1))
    return points.T #Nx11

# Add color based on label
def add_color_to_points(points):
    colors = []
    #print(points[:,4])
    for point in points:
        label = int(point[4])  # label is at index 4
        # Get color from map, default to black if label not found
        bgr_color = label_color_map.get(label, [255, 0, 0])
        # Convert BGR to RGB
        rgb_color = [bgr_color[2]/255.0, bgr_color[1]/255.0, bgr_color[0]/255.0]
        colors.append(rgb_color)
    return np.array(colors)


# Main processing
def process_and_visualize(file_path):
    # Read points
    points = read_bin_file(file_path)
    
    # Add colors
    colors = add_color_to_points(points)

    create_legend_image(label_color_map)
    
    # Visualize
    visualize_point_cloud_hr(points, colors, save_path="point_cloud_theirs_hr.png", width=1024, height=768)

# Example usage
if __name__ == "__main__":
    file_path = "submaps/01/000000.bin"  # Replace with your file path
    process_and_visualize(file_path)


(1888040,)
Legend saved to legend.png
Saved image to point_cloud_theirs_hr.png
